## Get Raw Data

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

In [2]:
#accesses local file
# csv_file = "Resources/time_series_19-covid-Confirmed.csv"
# raw_covid_df = pd.read_csv(csv_file).sort_values('Country/Region').reset_index(drop=True)

# direct URL access for up-to-date info
# non-raw data: https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv

url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
raw_covid_df = pd.read_csv(url).sort_values('Country/Region').reset_index(drop=True)

In [3]:
#Cleaning: remove parens from country names
raw_covid_df['Country/Region'] = raw_covid_df['Country/Region'].str.replace('(',"").str.replace(")","")
raw_covid_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,7,7,7,11,16,21,22,22,22,24
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,12,23,33,38,42,51,55,59,64,70
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,20,24,26,37,48,54,60,74,87,90
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,1,1,1,1,1,2,39,39,53,75
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## Separating Databases

In [4]:
#locale Index DB
index=np.arange(len(raw_covid_df))
raw_covid_df.insert(0,'Locale Index',index)

locale_index = pd.DataFrame()
locale_index['lat'] = raw_covid_df['Lat']
locale_index['long'] = raw_covid_df['Long']
locale_index['index'] = raw_covid_df['Locale Index']

In [82]:
#States DB
states_dirty = raw_covid_df[~raw_covid_df["Province/State"].str.contains(',', na=False)]
states = states_dirty[states_dirty['Province/State'].notna()]
# states.head()

,Locale Index,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20
8,8,Tasmania,Australia,-41.4545,145.9707,0,0,0,0,0,...,1,1,2,2,2,3,3,5,5,6
9,9,From Diamond Princess,Australia,35.4437,139.6380,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,10,Northern Territory,Australia,-12.4634,130.8456,0,0,0,0,0,...,0,0,0,0,1,1,1,1,1,1
11,11,South Australia,Australia,-34.9285,138.6007,0,0,0,0,0,...,7,7,7,7,7,9,9,16,19,20
12,12,Queensland,Australia,-28.0167,153.4000,0,0,0,0,0,...,13,13,15,15,18,20,20,35,46,61


In [83]:
#Cities DB (only in the US for now)
cities = raw_covid_df[raw_covid_df["Province/State"].str.contains(',', na=False)]
# cities.head()

,Locale Index,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20
193,193,"Placer County, CA",US,39.0916,-120.8039,0,0,0,0,0,...,5,5,5,7,0,0,0,0,0,0
194,194,"Orange County, CA",US,33.7879,-117.8531,0,0,0,0,1,...,3,3,3,4,0,0,0,0,0,0
195,195,"Norfolk County, MA",US,42.1767,-71.1449,0,0,0,0,0,...,2,2,6,6,0,0,0,0,0,0
196,196,"Maricopa County, AZ",US,33.2918,-112.4291,0,0,0,0,1,...,2,2,2,2,0,0,0,0,0,0
197,197,"Wake County, NC",US,35.8032,-78.5661,0,0,0,0,0,...,1,1,1,1,0,0,0,0,0,0


In [89]:
#Countries Only DB

#remove cities (as keeping cities would create case redundancies)
covid_countries = raw_covid_df[~raw_covid_df["Province/State"].str.contains(',', na=False)]

#use groupby to combine regions/states into country total, drop lat and long
covid_countries = covid_df.groupby(['Country/Region']).sum().drop(columns=['Lat','Long'])

covid_countries.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,1,1,4,4,5,7,7,7,11,16
Albania,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,10,12,23,33,38,42
Algeria,0,0,0,0,0,0,0,0,0,0,...,17,17,19,20,20,20,24,26,37,48
Andorra,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Antigua and Barbuda,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1


## Begin SQL Connection

In [31]:
rds_connection_string = "postgres:postgres@localhost:5432/covid_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [32]:
covid_countries.to_sql(name='covid_country', con=engine, if_exists='replace',index=True)
#if using constraints in SQL, using replace would not work. 

In [33]:
engine.table_names()

['covid_country']

In [36]:
pd.read_sql_query('select * from covid_country', con=engine).head()

,index,Afghanistan,Albania,Algeria,Andorra,Antigua and Barbuda,Argentina,Armenia,Aruba,Australia,...,Turkey,US,Ukraine,United Arab Emirates,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,occupied Palestinian territory
0,1/22/20,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1/23/20,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,2,0
2,1/24/20,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,2,0
3,1/25/20,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,2,0
4,1/26/20,0,0,0,0,0,0,0,0,4,...,0,5,0,0,0,0,0,0,2,0
